In [26]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_excel('data4(폐지사포함).xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19989 entries, 0 to 19988
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   회사명                             19989 non-null  object 
 1   거래소코드                           19989 non-null  int64  
 2   회계년도                            19989 non-null  object 
 3   [제조]총자본순이익률                     19489 non-null  float64
 4   [제조]총자본영업이익률                    19489 non-null  float64
 5   [제조]자기자본순이익률                    19489 non-null  float64
 6   [제조]매출액총이익률                     19489 non-null  float64
 7   [제조]매출액영업이익률                    19489 non-null  float64
 8   [제조]금융비용부담률                     19489 non-null  float64
 9   [제조]자기자본배율                      19489 non-null  float64
 10  [제조]부채비율                        19489 non-null  float64
 11  [제조]유동비율                        19489 non-null  float64
 12  [제조]당좌비율                        

In [52]:
data2 = pd.read_excel('폐지사추가1207.xlsx',header = 0)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   거래소코드       281 non-null    int64  
 1   부도여부        281 non-null    float64
 2   폐지사유        65 non-null     object 
 3   Unnamed: 3  16 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.9+ KB


In [53]:
data2.head(5)

,거래소코드,부도여부,폐지사유,Unnamed: 3
0,360,1.0,자본잠식율 50% 이상,NaN
1,830,0.0,회사합병,NaN
2,1300,0.0,회사합병,NaN
3,1310,1.0,NaN,NaN
4,1370,0.0,NaN,NaN


In [54]:
# 폐지사에서 부도여부를 확실시한 데이터
data2.drop(columns = {'폐지사유', 'Unnamed: 3'}, axis = 1, inplace = True )
data2.거래소코드 = data2.거래소코드.astype('int')
data2.거래소코드 = data2.거래소코드.map('{:06d}'.format)

data2.head(5)

,거래소코드,부도여부
0,000360,1.0
1,000830,0.0
2,001300,0.0
3,001310,1.0
4,001370,0.0


In [55]:
# 나머지 폐지되지 않은 데이터와 합침
data.거래소코드 = data.거래소코드.astype('int')
data.거래소코드 = data.거래소코드.map('{:06d}'.format)
data_code = pd.unique(data['거래소코드'])
data_code = pd.DataFrame(data_code)
data_code.rename(columns = {0 : '거래소코드'}, inplace = True)

merge_data = pd.merge(data_code, data2, how = 'outer', on = '거래소코드')
merge_data

,거래소코드,부도여부
0,058820,NaN
1,000990,NaN
2,050120,NaN
3,015940,0.0
4,045820,0.0
...,...,...
2208,205470,NaN
2209,145020,NaN
2210,069260,NaN
2211,024060,NaN


In [56]:
merge_data['부도여부'].value_counts(dropna=False)

NaN    1932
1.0     190
0.0      91
Name: 부도여부, dtype: int64

In [57]:
# 폐지되지 않은 데이터는 부도되지 않았으므로 모두 0 처리
normal_data = merge_data[merge_data['부도여부'].isna()]
normal_data.fillna(0, inplace = True)
normal_data

,거래소코드,부도여부
0,058820,0.0
1,000990,0.0
2,050120,0.0
5,023150,0.0
6,002360,0.0
...,...,...
2208,205470,0.0
2209,145020,0.0
2210,069260,0.0
2211,024060,0.0


In [58]:
# 전체 데이터에 합치기
# 1)normal data
data_f = pd.merge(data, normal_data, how = 'outer', on = '거래소코드')
data_f

,회사명,거래소코드,회계년도,[제조]총자본순이익률,[제조]총자본영업이익률,[제조]자기자본순이익률,[제조]매출액총이익률,[제조]매출액영업이익률,[제조]금융비용부담률,[제조]자기자본배율,...,[제조] 이자비용(천원),[제조]자본(*)(천원),[제조] 비유동자산(*)(천원),[제조] 현금 및 현금성자산(*)(천원),[제조] 유동부채(*)(천원),[제조]영업활동으로 인한 현금흐름(*)(천원),[제조]매출액(영업수익)(*)(천원),[제조] 당기순이익(손실)(천원),상장폐지일,부도여부
0,(주)CMG제약,058820,2006/12,-37.25,-12.93,-65.10,3.74,-35.59,-1.80,286.47,...,0.0,7058589.0,3132701.0,2850442.0,2971341.0,-2762580.0,3324793.0,0.0,NaN,0.0
1,(주)CMG제약,058820,2007/12,-5.55,-8.31,-6.41,17.48,-74.30,-22.46,327.16,...,0.0,20701209.0,13122035.0,2714192.0,1372494.0,-4659853.0,1795969.0,-890098.0,NaN,0.0
2,(주)CMG제약,058820,2008/12,-16.46,-13.37,-20.50,36.60,-35.02,4.18,316.67,...,0.0,24753170.0,17831672.0,6233270.0,2971345.0,-6283081.0,10808242.0,-4659443.0,NaN,0.0
3,(주)CMG제약,058820,2009/12,-63.34,-30.14,-106.40,24.33,-85.24,9.00,113.51,...,0.0,9412145.0,13008205.0,1062978.0,9050137.0,-5683304.0,10146823.0,-18176776.0,NaN,0.0
4,(주)CMG제약,058820,2010/12,-32.57,-31.74,-57.61,23.74,-65.66,8.87,83.25,...,0.0,17139590.0,12523986.0,2447346.0,3783230.0,-5983975.0,11350419.0,-7648552.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19984,흥아해운(주),003280,2009/12,-3.12,-1.65,-16.74,1.72,-1.43,2.60,339.46,...,0.0,120021672.0,387848693.0,11446397.0,164568752.0,-1306236.0,547635201.0,-14770940.0,NaN,0.0
19985,흥아해운(주),003280,2008/12,0.63,0.36,4.52,3.31,0.24,2.11,475.85,...,0.0,56420256.0,369145263.0,19183233.0,173841120.0,-22360291.0,600617536.0,2536260.0,NaN,0.0
19986,흥아해운(주),003280,2007/12,-2.75,-1.43,-14.66,3.12,-0.95,1.66,471.40,...,0.0,55892415.0,283901405.0,6292399.0,176525708.0,14901048.0,483349366.0,-8879104.0,NaN,0.0
19987,흥아해운(주),003280,2006/12,-0.55,-2.37,-2.48,2.61,-1.43,1.69,550.33,...,0.0,65251062.0,241054248.0,5385767.0,139963927.0,2398640.0,499319268.0,0.0,NaN,0.0


In [60]:
data_f.부도여부.value_counts(dropna = False)

0.0    18559
NaN     1430
Name: 부도여부, dtype: int64

In [61]:
# 2) 부도 데이터 합치기
a = pd.merge(data_f, data2, how = 'left', on = '거래소코드')
a

,회사명,거래소코드,회계년도,[제조]총자본순이익률,[제조]총자본영업이익률,[제조]자기자본순이익률,[제조]매출액총이익률,[제조]매출액영업이익률,[제조]금융비용부담률,[제조]자기자본배율,...,[제조]자본(*)(천원),[제조] 비유동자산(*)(천원),[제조] 현금 및 현금성자산(*)(천원),[제조] 유동부채(*)(천원),[제조]영업활동으로 인한 현금흐름(*)(천원),[제조]매출액(영업수익)(*)(천원),[제조] 당기순이익(손실)(천원),상장폐지일,부도여부_x,부도여부_y
0,(주)CMG제약,058820,2006/12,-37.25,-12.93,-65.10,3.74,-35.59,-1.80,286.47,...,7058589.0,3132701.0,2850442.0,2971341.0,-2762580.0,3324793.0,0.0,NaN,0.0,NaN
1,(주)CMG제약,058820,2007/12,-5.55,-8.31,-6.41,17.48,-74.30,-22.46,327.16,...,20701209.0,13122035.0,2714192.0,1372494.0,-4659853.0,1795969.0,-890098.0,NaN,0.0,NaN
2,(주)CMG제약,058820,2008/12,-16.46,-13.37,-20.50,36.60,-35.02,4.18,316.67,...,24753170.0,17831672.0,6233270.0,2971345.0,-6283081.0,10808242.0,-4659443.0,NaN,0.0,NaN
3,(주)CMG제약,058820,2009/12,-63.34,-30.14,-106.40,24.33,-85.24,9.00,113.51,...,9412145.0,13008205.0,1062978.0,9050137.0,-5683304.0,10146823.0,-18176776.0,NaN,0.0,NaN
4,(주)CMG제약,058820,2010/12,-32.57,-31.74,-57.61,23.74,-65.66,8.87,83.25,...,17139590.0,12523986.0,2447346.0,3783230.0,-5983975.0,11350419.0,-7648552.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19991,흥아해운(주),003280,2009/12,-3.12,-1.65,-16.74,1.72,-1.43,2.60,339.46,...,120021672.0,387848693.0,11446397.0,164568752.0,-1306236.0,547635201.0,-14770940.0,NaN,0.0,NaN
19992,흥아해운(주),003280,2008/12,0.63,0.36,4.52,3.31,0.24,2.11,475.85,...,56420256.0,369145263.0,19183233.0,173841120.0,-22360291.0,600617536.0,2536260.0,NaN,0.0,NaN
19993,흥아해운(주),003280,2007/12,-2.75,-1.43,-14.66,3.12,-0.95,1.66,471.40,...,55892415.0,283901405.0,6292399.0,176525708.0,14901048.0,483349366.0,-8879104.0,NaN,0.0,NaN
19994,흥아해운(주),003280,2006/12,-0.55,-2.37,-2.48,2.61,-1.43,1.69,550.33,...,65251062.0,241054248.0,5385767.0,139963927.0,2398640.0,499319268.0,0.0,NaN,0.0,NaN


In [62]:
a.부도여부_x.value_counts(dropna = False)

0.0    18559
NaN     1437
Name: 부도여부_x, dtype: int64

In [63]:
a.부도여부_y.value_counts(dropna = False)

NaN    18559
1.0      959
0.0      478
Name: 부도여부_y, dtype: int64

In [64]:
a['부도여부_x'].fillna(a['부도여부_y'], inplace = True)
a.부도여부_x.value_counts(dropna = False)

0.0    19037
1.0      959
Name: 부도여부_x, dtype: int64

In [65]:
data_final = a.drop('부도여부_y', axis = 1)
data_final.rename(columns = {'부도여부_x' : '부도여부'}, inplace = True)
data_final

,회사명,거래소코드,회계년도,[제조]총자본순이익률,[제조]총자본영업이익률,[제조]자기자본순이익률,[제조]매출액총이익률,[제조]매출액영업이익률,[제조]금융비용부담률,[제조]자기자본배율,...,[제조] 이자비용(천원),[제조]자본(*)(천원),[제조] 비유동자산(*)(천원),[제조] 현금 및 현금성자산(*)(천원),[제조] 유동부채(*)(천원),[제조]영업활동으로 인한 현금흐름(*)(천원),[제조]매출액(영업수익)(*)(천원),[제조] 당기순이익(손실)(천원),상장폐지일,부도여부
0,(주)CMG제약,058820,2006/12,-37.25,-12.93,-65.10,3.74,-35.59,-1.80,286.47,...,0.0,7058589.0,3132701.0,2850442.0,2971341.0,-2762580.0,3324793.0,0.0,NaN,0.0
1,(주)CMG제약,058820,2007/12,-5.55,-8.31,-6.41,17.48,-74.30,-22.46,327.16,...,0.0,20701209.0,13122035.0,2714192.0,1372494.0,-4659853.0,1795969.0,-890098.0,NaN,0.0
2,(주)CMG제약,058820,2008/12,-16.46,-13.37,-20.50,36.60,-35.02,4.18,316.67,...,0.0,24753170.0,17831672.0,6233270.0,2971345.0,-6283081.0,10808242.0,-4659443.0,NaN,0.0
3,(주)CMG제약,058820,2009/12,-63.34,-30.14,-106.40,24.33,-85.24,9.00,113.51,...,0.0,9412145.0,13008205.0,1062978.0,9050137.0,-5683304.0,10146823.0,-18176776.0,NaN,0.0
4,(주)CMG제약,058820,2010/12,-32.57,-31.74,-57.61,23.74,-65.66,8.87,83.25,...,0.0,17139590.0,12523986.0,2447346.0,3783230.0,-5983975.0,11350419.0,-7648552.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19991,흥아해운(주),003280,2009/12,-3.12,-1.65,-16.74,1.72,-1.43,2.60,339.46,...,0.0,120021672.0,387848693.0,11446397.0,164568752.0,-1306236.0,547635201.0,-14770940.0,NaN,0.0
19992,흥아해운(주),003280,2008/12,0.63,0.36,4.52,3.31,0.24,2.11,475.85,...,0.0,56420256.0,369145263.0,19183233.0,173841120.0,-22360291.0,600617536.0,2536260.0,NaN,0.0
19993,흥아해운(주),003280,2007/12,-2.75,-1.43,-14.66,3.12,-0.95,1.66,471.40,...,0.0,55892415.0,283901405.0,6292399.0,176525708.0,14901048.0,483349366.0,-8879104.0,NaN,0.0
19994,흥아해운(주),003280,2006/12,-0.55,-2.37,-2.48,2.61,-1.43,1.69,550.33,...,0.0,65251062.0,241054248.0,5385767.0,139963927.0,2398640.0,499319268.0,0.0,NaN,0.0


In [66]:
data_final.sort_values(by = ['거래소코드', '회계년도'], ascending = True,
                      inplace = True)
data_1 = data_final[data_final['부도여부']==1]
data_1[['거래소코드', '회계년도', '부도여부']]

,거래소코드,회계년도,부도여부
15062,000360,2006/12,1.0
15061,000360,2007/12,1.0
15058,000360,2008/12,1.0
15057,000360,2009/12,1.0
15054,000360,2010/12,1.0
...,...,...,...
7261,160350,2014/12,1.0
15392,179280,2011/12,1.0
15391,179280,2012/12,1.0
15389,179280,2013/12,1.0


In [68]:
# 가장 최근 회계년도만을 부도처리하고 그 이외는 정상처리
budo_list = list(data_1['거래소코드'].unique())
for i in budo_list:
    index_list = list(data_final[data_final['거래소코드']==i].index)
    for index in index_list:
        if index == index_list[-1]:
            data_final.loc[index, '부도여부'] = 1
        else :
            data_final.loc[index, '부도여부'] = 0

In [69]:
data_final.부도여부.value_counts()

0.0    19807
1.0      189
Name: 부도여부, dtype: int64

In [70]:
print(len(data_final[data_final['부도여부'] == 1].거래소코드.unique()))
print(len(data_final[data_final['부도여부'] == 0].거래소코드.unique()))


189
2208


In [71]:
data_final

,회사명,거래소코드,회계년도,[제조]총자본순이익률,[제조]총자본영업이익률,[제조]자기자본순이익률,[제조]매출액총이익률,[제조]매출액영업이익률,[제조]금융비용부담률,[제조]자기자본배율,...,[제조] 이자비용(천원),[제조]자본(*)(천원),[제조] 비유동자산(*)(천원),[제조] 현금 및 현금성자산(*)(천원),[제조] 유동부채(*)(천원),[제조]영업활동으로 인한 현금흐름(*)(천원),[제조]매출액(영업수익)(*)(천원),[제조] 당기순이익(손실)(천원),상장폐지일,부도여부
13920,동화약품(주),000020,2006/03,3.95,8.06,4.99,48.36,10.24,-0.62,557.53,...,0.0,155725978.0,62083281.0,12195978.0,24505360.0,26246911.0,152821587.0,0.0,NaN,0.0
13917,동화약품(주),000020,2007/03,4.13,7.26,5.09,48.87,9.53,-1.49,576.40,...,0.0,160998236.0,65095476.0,9428619.0,26052899.0,23185889.0,148738623.0,0.0,NaN,0.0
13918,동화약품(주),000020,2008/03,10.28,16.04,12.64,55.66,19.16,-2.06,641.36,...,0.0,179141892.0,84053277.0,4677396.0,32920471.0,23655312.0,175119773.0,21495381.0,NaN,0.0
13919,동화약품(주),000020,2009/03,10.72,15.03,14.83,55.77,21.19,-1.75,735.24,...,0.0,205363087.0,183993477.0,10967304.0,80426807.0,31155090.0,188607515.0,28518530.0,NaN,0.0
13913,동화약품(주),000020,2010/12,1.68,3.18,2.54,43.57,4.79,0.68,765.86,...,0.0,213915026.0,202168003.0,18140995.0,82657565.0,18587828.0,215279424.0,5427207.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6136,(주)엑세스바이오인코퍼레이션,950130,2014/12,5.50,6.45,6.45,47.82,12.58,-0.72,3831.33,...,0.0,57093892.0,22775157.0,2277486.0,7884320.0,-1112479.0,32387575.0,3624609.0,NaN,0.0
6132,(주)엑세스바이오인코퍼레이션,950130,2015/12,3.31,2.91,3.70,30.81,5.58,-0.48,3850.51,...,0.0,61665064.0,33716662.0,2571609.0,1928812.0,5762413.0,34758550.0,2276717.0,NaN,0.0
6135,(주)엑세스바이오인코퍼레이션,950130,2016/12,1.60,1.97,2.08,41.58,5.36,0.00,3824.26,...,0.0,63152029.0,50080698.0,22694029.0,2419545.0,6556892.0,29762933.0,1352476.0,NaN,0.0
17526,잉글우드랩(주),950140,2015/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,25980689.0,25579784.0,1137.0,476282.0,-2379.0,0.0,400905.0,NaN,0.0


In [72]:
# 회계년도와 상장폐지년도 같은 경우, 상장폐지년도의 전년도도 부도 처리
data_final.reset_index(inplace = True)
data_final

,index,회사명,거래소코드,회계년도,[제조]총자본순이익률,[제조]총자본영업이익률,[제조]자기자본순이익률,[제조]매출액총이익률,[제조]매출액영업이익률,[제조]금융비용부담률,...,[제조] 이자비용(천원),[제조]자본(*)(천원),[제조] 비유동자산(*)(천원),[제조] 현금 및 현금성자산(*)(천원),[제조] 유동부채(*)(천원),[제조]영업활동으로 인한 현금흐름(*)(천원),[제조]매출액(영업수익)(*)(천원),[제조] 당기순이익(손실)(천원),상장폐지일,부도여부
0,13920,동화약품(주),000020,2006/03,3.95,8.06,4.99,48.36,10.24,-0.62,...,0.0,155725978.0,62083281.0,12195978.0,24505360.0,26246911.0,152821587.0,0.0,NaN,0.0
1,13917,동화약품(주),000020,2007/03,4.13,7.26,5.09,48.87,9.53,-1.49,...,0.0,160998236.0,65095476.0,9428619.0,26052899.0,23185889.0,148738623.0,0.0,NaN,0.0
2,13918,동화약품(주),000020,2008/03,10.28,16.04,12.64,55.66,19.16,-2.06,...,0.0,179141892.0,84053277.0,4677396.0,32920471.0,23655312.0,175119773.0,21495381.0,NaN,0.0
3,13919,동화약품(주),000020,2009/03,10.72,15.03,14.83,55.77,21.19,-1.75,...,0.0,205363087.0,183993477.0,10967304.0,80426807.0,31155090.0,188607515.0,28518530.0,NaN,0.0
4,13913,동화약품(주),000020,2010/12,1.68,3.18,2.54,43.57,4.79,0.68,...,0.0,213915026.0,202168003.0,18140995.0,82657565.0,18587828.0,215279424.0,5427207.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19991,6136,(주)엑세스바이오인코퍼레이션,950130,2014/12,5.50,6.45,6.45,47.82,12.58,-0.72,...,0.0,57093892.0,22775157.0,2277486.0,7884320.0,-1112479.0,32387575.0,3624609.0,NaN,0.0
19992,6132,(주)엑세스바이오인코퍼레이션,950130,2015/12,3.31,2.91,3.70,30.81,5.58,-0.48,...,0.0,61665064.0,33716662.0,2571609.0,1928812.0,5762413.0,34758550.0,2276717.0,NaN,0.0
19993,6135,(주)엑세스바이오인코퍼레이션,950130,2016/12,1.60,1.97,2.08,41.58,5.36,0.00,...,0.0,63152029.0,50080698.0,22694029.0,2419545.0,6556892.0,29762933.0,1352476.0,NaN,0.0
19994,17526,잉글우드랩(주),950140,2015/12,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,25980689.0,25579784.0,1137.0,476282.0,-2379.0,0.0,400905.0,NaN,0.0


In [73]:
# 회계년도 연도만 남김
try :
    data_final['회계년도'] = data_final['회계년도'].str.split('/').str[0]
except ValueError:
    ;

In [74]:
# 상장폐지 연도만 남김
try :
    data_final['상장폐지일'] = data_final['상장폐지일'].str.split('/').str[0]
except ValueError :
    ;

In [75]:
same_year = data_final[(data_final['회계년도']==data_final['상장폐지일']) & (data_final['부도여부']==1)]
same_year = same_year[same_year['회계년도']!='2006'] #2006년은 전년도가 존재하지 않으므로 제외
idx = same_year.index
idx

Int64Index([2959, 3671, 8275, 8474, 8732, 14321], dtype='int64')

In [76]:
same_year[['회계년도', '상장폐지일']]

,회계년도,상장폐지일
2959,2009,2009
3671,2012,2012
8275,2009,2009
8474,2009,2009
8732,2009,2009
14321,2013,2013


In [77]:
# 부도 전년도 데이터 유무 확인
data_final.loc[idx-1][['회계년도', '상장폐지일', '부도여부']]

,회계년도,상장폐지일,부도여부
2958,2008,2009,0.0
3670,2011,2012,0.0
8274,2007,2009,0.0
8473,2008,2009,0.0
8731,2008,2009,0.0
14320,2012,2013,0.0


In [78]:
data_final.loc[8273:8275][['회사명', '회계년도', '상장폐지일', '부도여부']]

,회사명,회계년도,상장폐지일,부도여부
8273,(주)소예,2006,2009,0.0
8274,(주)소예,2007,2009,0.0
8275,(주)소예,2009,2009,1.0


In [81]:
# 부도 전년도 데이터가 없으므로 제외

same_year = same_year[same_year['회사명']!='(주)소예']
idx = same_year.index
idx

Int64Index([2959, 3671, 8474, 8732, 14321], dtype='int64')

In [84]:
# 나머지 데이터의 부도 전년도 데이터 부도 처리
data_final.loc[idx-1, '부도여부'] = 1
data_final.loc[idx-1][['회계년도', '상장폐지일', '부도여부']]

,회계년도,상장폐지일,부도여부
2958,2008,2009,1.0
3670,2011,2012,1.0
8473,2008,2009,1.0
8731,2008,2009,1.0
14320,2012,2013,1.0


In [86]:
# 회생 후 재 상장폐지 된 기업 재처리
g = data_final.groupby(['거래소코드'])['상장폐지일'].nunique().reset_index()
g[g['상장폐지일']>=2]

,거래소코드,상장폐지일
640,022520,2


In [87]:
data_final[data_final['거래소코드']=='022520'][['회사명', '거래소코드', '회계년도'
                                          , '상장폐지일', '부도여부']]

,회사명,거래소코드,회계년도,상장폐지일,부도여부
6397,코오롱아이넷(주),022520,2006,2011,0.0
6398,코오롱아이넷(주),022520,2006,2012,0.0
6399,코오롱아이넷(주),022520,2007,2012,0.0
6400,코오롱아이넷(주),022520,2007,2011,0.0
6401,코오롱아이넷(주),022520,2008,2011,0.0
6402,코오롱아이넷(주),022520,2008,2012,0.0
6403,코오롱아이넷(주),022520,2009,2012,0.0
6404,코오롱아이넷(주),022520,2009,2011,0.0
6405,코오롱아이넷(주),022520,2010,2012,0.0
6406,코오롱아이넷(주),022520,2010,2011,0.0


In [88]:
# 합병 기업으로 확인되어 재라벨링 하지 않음
data_final.to_csv('budo_labeling_완.csv')